# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [49]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [52]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [53]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [54]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [55]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [56]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query1 = """select artist_name, song_title, song_length \
            from event_song_1 \
            where session_id = 338 and item_in_session = 4"""

# create table based on the query1
query = "CREATE TABLE IF NOT EXISTS event_song_1 "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length double, PRIMARY KEY (session_id, item_in_session))"

# Primary key selection for query 1
# from example of data: the primary key of session_id and item_in_session should be enough because the item count \
# in each session should not be duplicate

try:
    session.execute(query)
except Exception as e:
    print(e)

In [57]:
# insert data from event_datafile_new.csv to cassandra
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO event_song_1 (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [58]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

df_result1 = pd.DataFrame(columns=['artist_name', 'song_title', 'song_length'])
for i, row in enumerate(rows):
    df_result1.loc[i] = [row.artist_name, row.song_title, row.song_length]

In [59]:
df_result1

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [60]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
query2 = """select artist_name, song_title, first_name, last_name \
            from event_song_2 \
            where user_id = 10 and session_id = 182"""

# create table based on query2
query_create_2 = "CREATE TABLE IF NOT EXISTS event_song_2 "
query_create_2 = query_create_2 + "(user_id int, session_id int, item_in_session int, artist_name text, \
song_title text, first_name text, last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"

# For the second query, I decided to choose add 1 more column for primary key, i think it's saver than the first query

try:
    session.execute(query_create_2)
except Exception as e:
    print(e)

In [61]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO event_song_2 (user_id, session_id, item_in_session, artist_name, \
        song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [62]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

df_result2 = pd.DataFrame(columns=['artist_name', 'song_title', 'first_name', 'last_name'])
for i, row in enumerate(rows):
    df_result2.loc[i] = [row.artist_name, row.song_title, row.first_name, row.last_name]

In [63]:
df_result2

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [64]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """select first_name, last_name \
            from event_song_3 \
            where song_title = 'All Hands Against His Own'"""

# create table based on query3
query_create_3 = "CREATE TABLE IF NOT EXISTS event_song_3 "
query_create_3 = query_create_3 + "(song_title text, user_id int, session_id int, item_in_session int, \
                first_name text, last_name text, PRIMARY KEY (song_title, user_id, session_id, item_in_session))"

# The primary key for this query is the same as second query

try:
    session.execute(query_create_3)
except Exception as e:
    print(e)        

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO event_song_3 (song_title, user_id, session_id, item_in_session, \
        first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))

In [66]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

df_result3 = pd.DataFrame(columns=['first_name', 'last_name'])
for i, row in enumerate(rows):
    df_result3.loc[i] = [row.first_name, row.last_name]

In [67]:
df_result3

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [68]:
drop_table_list = ['event_song_1', 'event_song_2', 'event_song_3']
for table_name in drop_table_list:
    query = f"drop table {table_name}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [69]:
session.shutdown()
cluster.shutdown()